## Step 26: 계산 그래프 시각화 (2)

DeZero에서 실행한 계산을 DOT 언어로 변환하는 기능을 구현해보자.

### 26.2 계산 그래프에서 DOT 언어로 변환하기

```python
# dezero/utils.py: var to dot
def _dot_var(v, verbose=False):  # 로컬에서만 사용할 함수: 외부에서 부를 수 없다
    dot_var = '{} [label="{}", color=orange, style=filled]\n'  # 변수 노드 프레임

    name = '' if v.name is None else v.name

    if verbose and v.data is not None:
        if v.name is not None:
            name += ': '
        name += str(v.shape) + ' ' + str(v.dtype)
    return dot_var.format(id(v), name)
```


사용 예시)
```python
x = Variable(np.random.randn(2,3))
x.name = 'x'

print(_dot_var(x))
print(_dot_var(x, verbose=True))
```

변수 노드의 ID는 파이썬 내장함수 id()에 의해 고유하게 지정된다.

verbose=True로 설정 시 name에 ndarray 인스턴스 shape와 type이 추가된다.

```python
# dezero/utils.py: func to dot
def _dot_func(f):
    dot_func = '{} [label="{}", color=lightblue, style=filled, shape=box]\n'
    txt = dot_func.format(id(f), f.__class__.__name__)  # 계산 함수 클래스명을 노드 name으로

    dot_edge = '{} -> {}\n'
    for x in f.inputs:
        txt += dot_edge.format(id(x), id(f))
    for y in f.outputs:
        txt += dot_edge.format(id(x), id(y()))  # y는 weakref
    return txt
```


사용 예시)
```python
x0 = Variable(np.random.randn(1.0))
x1 = Variable(np.random.randn(1.0))
y = x0 + x1
txt = _dot_func(y.creator)

print(txt)
```

최종은 다음과 같다.

```python
def get_dot_graph(output, verbose=True):
    txt = ''
    funcs = []
    seen_set = set()

    def add_func(f):
        if f not in seen_set:
            funcs.append(f)
            funcs.sort(key=lambda x: x.generation)
            seen_set.add(f)

        add_func(output.creator)
        txt += _dot_var(output, verbose)

        while funcs:
            func = funcs.pop()
            txt += _dot_func(func)
            for x in func.inputs:
                txt += _dot_var(x, verbose)

                if x.creator is not None:
                    add_func(x.creator)
        return 'digraph g{\n' + txt + '}'
```

### 26.3 이미지 변환까지 한 번에

위 함수로 계산 그래프를 DOT 언어로 표현하는 것까지 완료했으나, 지금은 이를 실행 후 따로 터미널에서 이미지 변환을 해주어야 한다. 따라서 이 과정까지 한 번에 할 수 있도록 하겠다.

```python
def plot_dot_graph(output, verbose=True, to_file='graph.png'):
    dot_graph = get_dot_graph(output, verbose)

    # dot 데이터를 파일에 저장
    tmp_dir = os.path.join(os.path.expanduser('~'), '.dezero')
    if not os.path.exists(tmp_dir):
        os.mkdir(tmp_dir)
    graph_path = os.path.join(tmp_dir, 'tmp_graph.dot')

    with open(graph_path, 'w') as f:
        f.write(dot_graph)

    # dot 명령 호출
    extension = os.path.splitext(to_file)[1][1:]  # 확장자
    cmd = f'dot {graph_path} -T {extension} -o {to_file}'
    subprocess.run(cmd, shell=True)
```

### 26.4 동작 확인

```python
import numpy as np

from dezero import Variable
from dezero.utils import plot_dot_graph
from dezero.utils import get_dot_graph

from step24 import goldstein

x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = goldstein(x, y)
z.backward()

# print(get_dot_graph(z))

x.name = 'x'
y.name = 'y'
z.name = 'z'
plot_dot_graph(z, verbose=False,
               to_file='./Kangbeen/steps/step26.png')

```

![step26](https://user-images.githubusercontent.com/76294398/183808954-efe75bca-08e8-4608-9ab6-6752d8b35751.png)